In [1]:
!pip install nltk

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split,KFold,GridSearchCV,RandomizedSearchCV,cross_val_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB#Multinomial Naive Byes used for text classification or for discrete dataset while GaussianNB used for continuous data.
import nltk
from nltk.stem import PorterStemmer,LancasterStemmer,SnowballStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize


In [3]:
from google.colab import files
uploaded=files.upload()

Saving SMS_test.csv to SMS_test.csv
Saving SMS_train.csv to SMS_train.csv


In [4]:
df_train = pd.read_csv("SMS_train.csv", encoding='latin1')

In [5]:
df_test= pd.read_csv("SMS_test.csv", encoding='latin1')

In [6]:
df=pd.concat([df_train,df_test],ignore_index=False,axis=0)

In [7]:
df.head()

,S. No.,Message_body,Label
0,1,Rofl. Its true to its name,Non-Spam
1,2,The guy did some bitching but I acted like i'd...,Non-Spam
2,3,"Pity, * was in mood for that. So...any other s...",Non-Spam
3,4,Will ü b going to esplanade fr home?,Non-Spam
4,5,This is the 2nd time we have tried 2 contact u...,Spam


In [ ]:
df.shape

(1082, 3)

In [8]:
df=df.drop('S. No.',axis=1)

#TEXT PREPROCESSING STAGE-1
#1TOKENIZATION
#2LOWERCASE
#3STOPWORDS
#4STEMMING
#5LAMMITIZATION

In [9]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
import re

In [12]:
df['Message_body'] = df['Message_body'].str.lower()#Most prior step to convert all the words into lower case

In [13]:
df.sample(10)

,Message_body,Label
40,customer service annoncement. you have a new y...,Spam
618,"hi this is amy, we will be sending you a free ...",Spam
176,i dont know exactly could you ask chechi.,Non-Spam
14,no. i meant the calculation is the same. that ...,Non-Spam
887,sry da..jst nw only i came to home..,Non-Spam
44,private! your 2003 account statement for shows...,Spam
715,gain the rights of a wife.dont demand it.i am ...,Non-Spam
213,tell me something. thats okay.,Non-Spam
79,carlos is down but i have to pick it up from h...,Non-Spam
287,i have gone into get info bt dont know what to do,Non-Spam


In [14]:
stemmer = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))

In [15]:
def clean_text(text):
  words=re.sub('[^a-zA-Z]'," ",text).lower().split()
  words = [stemmer.stem(word) for word in words if word not in stop_words]
  cleaned_text = ' '.join(words)
  return cleaned_text

In [16]:
df['clean_stem'] = df['Message_body'].apply(clean_text)

In [24]:
df.head(10)

,Message_body,Label,clean_stem,clean_Lamit,y
0,rofl. its true to its name,Non-Spam,rofl true name,rofl true name,0
1,the guy did some bitching but i acted like i'd...,Non-Spam,guy bitch act like interest buy someth els nex...,guy bitching acted like interested buying some...,0
2,"pity, * was in mood for that. so...any other s...",Non-Spam,piti mood suggest,pity mood suggestion,0
3,will ü b going to esplanade fr home?,Non-Spam,b go esplanad fr home,b going esplanade fr home,0
4,this is the 2nd time we have tried 2 contact u...,Spam,nd time tri contact u u pound prize claim easi...,nd time tried contact u u pound prize claim ea...,1
5,reminder from o2: to get 2.50 pounds free call...,Spam,remind get pound free call credit detail great...,reminder get pound free call credit detail gre...,1
6,huh y lei...,Non-Spam,huh lei,huh lei,0
7,why don't you wait 'til at least wednesday to ...,Non-Spam,wait til least wednesday see get,wait til least wednesday see get,0
8,ard 6 like dat lor.,Non-Spam,ard like dat lor,ard like dat lor,0
9,ok lor... sony ericsson salesman... i ask shuh...,Non-Spam,ok lor soni ericsson salesman ask shuhui say q...,ok lor sony ericsson salesman ask shuhui say q...,0


In [18]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [19]:
def lemmatize_text(text):
    words = re.sub('[^a-zA-Z]', ' ', text).lower().split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    lemmatized_text = ' '.join(lemmatized_words)
    return lemmatized_text

In [20]:
df['clean_Lamit']=df['Message_body'].apply(lemmatize_text)

In [21]:
df['y'] = pd.Categorical(df['Label']).codes

In [22]:
def tokenize_text(text):
    return word_tokenize(text)

In [ ]:
df

,Message_body,Label,clean_stem,clean_Lamit,y
0,rofl. its true to its name,Non-Spam,rofl true name,rofl true name,0
1,the guy did some bitching but i acted like i'd...,Non-Spam,guy bitch act like interest buy someth els nex...,guy bitching acted like interested buying some...,0
2,"pity, * was in mood for that. so...any other s...",Non-Spam,piti mood suggest,pity mood suggestion,0
3,will ü b going to esplanade fr home?,Non-Spam,b go esplanad fr home,b going esplanade fr home,0
4,this is the 2nd time we have tried 2 contact u...,Spam,nd time tri contact u u pound prize claim easi...,nd time tried contact u u pound prize claim ea...,1
...,...,...,...,...,...
120,7 wonders in my world 7th you 6th ur style 5th...,Non-Spam,wonder world th th ur style th ur smile th ur ...,wonder world th th ur style th ur smile th ur ...,0
121,try to do something dear. you read something f...,Non-Spam,tri someth dear read someth exam,try something dear read something exam,0
122,sun ah... thk mayb can if dun have anythin on....,Non-Spam,sun ah thk mayb dun anythin thk book e lesson ...,sun ah thk mayb dun anythin thk book e lesson ...,0
123,"symptoms when u are in love: ""1.u like listeni...",Non-Spam,symptom u love u like listen song u get stop u...,symptom u love u like listening song u get sto...,0


In [ ]:
df

,Message_body,Label,clean_stem,clean_Lamit,y
0,rofl. its true to its name,Non-Spam,rofl true name,rofl true name,0
1,the guy did some bitching but i acted like i'd...,Non-Spam,guy bitch act like interest buy someth els nex...,guy bitching acted like interested buying some...,0
2,"pity, * was in mood for that. so...any other s...",Non-Spam,piti mood suggest,pity mood suggestion,0
3,will ü b going to esplanade fr home?,Non-Spam,b go esplanad fr home,b going esplanade fr home,0
4,this is the 2nd time we have tried 2 contact u...,Spam,nd time tri contact u u pound prize claim easi...,nd time tried contact u u pound prize claim ea...,1
...,...,...,...,...,...
120,7 wonders in my world 7th you 6th ur style 5th...,Non-Spam,wonder world th th ur style th ur smile th ur ...,wonder world th th ur style th ur smile th ur ...,0
121,try to do something dear. you read something f...,Non-Spam,tri someth dear read someth exam,try something dear read something exam,0
122,sun ah... thk mayb can if dun have anythin on....,Non-Spam,sun ah thk mayb dun anythin thk book e lesson ...,sun ah thk mayb dun anythin thk book e lesson ...,0
123,"symptoms when u are in love: ""1.u like listeni...",Non-Spam,symptom u love u like listen song u get stop u...,symptom u love u like listening song u get sto...,0


In [23]:
def tokenize_text(text):
        return word_tokenize(text)

In [25]:
df['tokenized_clean_lamit'] = df['clean_Lamit'].apply(tokenize_text)

In [26]:
df

,Message_body,Label,clean_stem,clean_Lamit,y,tokenized_clean_lamit
0,rofl. its true to its name,Non-Spam,rofl true name,rofl true name,0,"[rofl, true, name]"
1,the guy did some bitching but i acted like i'd...,Non-Spam,guy bitch act like interest buy someth els nex...,guy bitching acted like interested buying some...,0,"[guy, bitching, acted, like, interested, buyin..."
2,"pity, * was in mood for that. so...any other s...",Non-Spam,piti mood suggest,pity mood suggestion,0,"[pity, mood, suggestion]"
3,will ü b going to esplanade fr home?,Non-Spam,b go esplanad fr home,b going esplanade fr home,0,"[b, going, esplanade, fr, home]"
4,this is the 2nd time we have tried 2 contact u...,Spam,nd time tri contact u u pound prize claim easi...,nd time tried contact u u pound prize claim ea...,1,"[nd, time, tried, contact, u, u, pound, prize,..."
...,...,...,...,...,...,...
120,7 wonders in my world 7th you 6th ur style 5th...,Non-Spam,wonder world th th ur style th ur smile th ur ...,wonder world th th ur style th ur smile th ur ...,0,"[wonder, world, th, th, ur, style, th, ur, smi..."
121,try to do something dear. you read something f...,Non-Spam,tri someth dear read someth exam,try something dear read something exam,0,"[try, something, dear, read, something, exam]"
122,sun ah... thk mayb can if dun have anythin on....,Non-Spam,sun ah thk mayb dun anythin thk book e lesson ...,sun ah thk mayb dun anythin thk book e lesson ...,0,"[sun, ah, thk, mayb, dun, anythin, thk, book, ..."
123,"symptoms when u are in love: ""1.u like listeni...",Non-Spam,symptom u love u like listen song u get stop u...,symptom u love u like listening song u get sto...,0,"[symptom, u, love, u, like, listening, song, u..."


In [27]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
x = cv.fit_transform(df['clean_Lamit']).toarray()

In [28]:
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [29]:
y=df['y']

In [ ]:
y

0      0
1      0
2      0
3      0
4      1
      ..
120    0
121    0
122    0
123    0
124    0
Name: y, Length: 1082, dtype: int8

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 0)

In [31]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [32]:
y_pred=spam_detect_model.predict(X_test)

In [33]:
from sklearn.metrics import accuracy_score,classification_report

In [34]:
score=accuracy_score(y_test,y_pred)
print(score)

0.9585253456221198


In [35]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       165
           1       0.94      0.88      0.91        52

    accuracy                           0.96       217
   macro avg       0.95      0.93      0.94       217
weighted avg       0.96      0.96      0.96       217



In [36]:
# Creating the TFIDF model
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features=2500)
x = tv.fit_transform(df['clean_Lamit']).toarray()

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 0)

In [38]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [39]:
y_pred=spam_detect_model.predict(X_test)

In [40]:
score=accuracy_score(y_test,y_pred)
print(score)

0.9170506912442397


In [41]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      0.90      0.95       186
           1       0.63      1.00      0.78        31

    accuracy                           0.92       217
   macro avg       0.82      0.95      0.86       217
weighted avg       0.95      0.92      0.92       217



In [42]:
!pip install gensim

In [ ]:
import gensim.downloader as api
import gensim

In [ ]:
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
vec_king = wv['king']

In [ ]:
vec_king

array([ 1.25976562e-01,  2.97851562e-02,  8.60595703e-03,  1.39648438e-01,
       -2.56347656e-02, -3.61328125e-02,  1.11816406e-01, -1.98242188e-01,
        5.12695312e-02,  3.63281250e-01, -2.42187500e-01, -3.02734375e-01,
       -1.77734375e-01, -2.49023438e-02, -1.67968750e-01, -1.69921875e-01,
        3.46679688e-02,  5.21850586e-03,  4.63867188e-02,  1.28906250e-01,
        1.36718750e-01,  1.12792969e-01,  5.95703125e-02,  1.36718750e-01,
        1.01074219e-01, -1.76757812e-01, -2.51953125e-01,  5.98144531e-02,
        3.41796875e-01, -3.11279297e-02,  1.04492188e-01,  6.17675781e-02,
        1.24511719e-01,  4.00390625e-01, -3.22265625e-01,  8.39843750e-02,
        3.90625000e-02,  5.85937500e-03,  7.03125000e-02,  1.72851562e-01,
        1.38671875e-01, -2.31445312e-01,  2.83203125e-01,  1.42578125e-01,
        3.41796875e-01, -2.39257812e-02, -1.09863281e-01,  3.32031250e-02,
       -5.46875000e-02,  1.53198242e-02, -1.62109375e-01,  1.58203125e-01,
       -2.59765625e-01,  